In [6]:
import os
import sys
from pathlib import Path

# sys.path = ['', '/home/therring/Workspace/brainbot_ws/devel/lib/python3/dist-packages', '/home/therring/anaconda3/envs/brainbot/lib/python310.zip', '/home/therring/anaconda3/envs/brainbot/lib/python3.10', '/home/therring/anaconda3/envs/brainbot/lib/python3.10/lib-dynload', '/home/therring/.local/lib/python3.10/site-packages', '/home/therring/Workspace/costfuncgraph/src/costfuncgraph_python/src', '/home/therring/anaconda3/envs/brainbot/lib/python3.10/site-packages', '/home/therring/anaconda3/envs/brainbot/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg']

# The conda python3 has these in its path, idk why it doesn't work here but this fixes the bindings
# sys.path.insert(0 ,"/home/therring/Workspace/ompl/py-bindings")
# sys.path.insert(0 ,"/home/therring/Workspace/ompl/build/lib")

from ompl import base as ob
import grapeshot as gs

pybullet build time: Feb 24 2023 13:05:45


In [7]:
planner_name = "RRTConnect"
planning_time = 1
vis = False


In [8]:
robot_name = "fetch"
group_name = "arm"

In [9]:
script_path = ! pwd
script_path = (str)(script_path)
script_path = Path(script_path)

In [10]:
asset_path = script_path / ".." / "assets"
robot_path = asset_path / robot_name

In [12]:
world = gs.model.World(vis)
robot = world.add_robot(robot_path / f"{robot_name}.urdf", robot_path / f"{robot_name}.srdf")
env = world.add_environment_yaml(asset_path / "environments" / "shelf_simple.yml")
world.configure_acm()

context = gs.ompl.get_OMPL_context(world, [(robot, group_name)], planner_name)

while True:
    start = ob.State(context.ss)
    goal = ob.State(context.ss)

    while not context.vss.sample(start.get()):
        pass
    while not context.vss.sample(goal.get()):
        pass

    context.setup.setStartAndGoalStates(start, goal)
    solved = context.setup.solve(planning_time)

    if path := context.get_solution_path():
        for state in path.getStates():
            context.set_state(state)
            time.sleep(0.01)
    else:
        print("Failed to find exact solution. Displaying start and goal states.")
        context.set_state(start.get())
        input("Press enter to display goal.")
        context.set_state(goal.get())
        input("Press enter to continue.")

    context.clear_query()

ValueError: /home/therring/anaconda3/envs/brainbot/bin/python: undefined symbol: tdou